In [3]:
import numpy as np
import cv2
from skimage import img_as_float


def intuitionistic_fuzzy_membership(image, a=0.5, b=0.3):
    norm_image = img_as_float(image)
    mu = np.clip(norm_image, 0, 1)
    nu = 1 - mu
    pi = 1 - mu - nu
    return mu, nu, pi


def fuse_ifs(mu1, nu1, pi1, mu2, nu2, pi2, alpha=0.5):
    fused_mu = alpha * mu1 + (1 - alpha) * mu2
    fused_nu = alpha * nu1 + (1 - alpha) * nu2
    fused_pi = alpha * pi1 + (1 - alpha) * pi2

    fused_image = np.clip(fused_mu - fused_nu, 0, 1)
    return fused_image


def medical_image_fusion(image1, image2, alpha=0.5):
    image1_ycbcr = cv2.cvtColor(image1, cv2.COLOR_BGR2YCrCb)
    image2_ycbcr = cv2.cvtColor(image2, cv2.COLOR_BGR2YCrCb)

    image1_Y = image1_ycbcr[:, :, 0]
    image2_Y = image2_ycbcr[:, :, 0]

    mu1, nu1, pi1 = intuitionistic_fuzzy_membership(image1_Y)
    mu2, nu2, pi2 = intuitionistic_fuzzy_membership(image2_Y)

    fused_Y = fuse_ifs(mu1, nu1, pi1, mu2, nu2, pi2, alpha)

    fused_image_ycbcr = image1_ycbcr.copy()
    fused_image_ycbcr[:, :, 0] = np.uint8(fused_Y * 255)
    fused_image = cv2.cvtColor(fused_image_ycbcr, cv2.COLOR_YCrCb2BGR)

    return fused_image


image1 = cv2.imread("data/fusion/052 (2).png")
image2 = cv2.imread("data/fusion/052.png")

fused_image = medical_image_fusion(image1, image2, alpha=0.1)

cv2.imwrite("fused_image_fuzzy.png", fused_image)

True